# Example: End-to-end feature store usage


In [10]:
from datetime import datetime
from snowflake_feature_store.examples import *
from snowflake_feature_store.connection import get_connection
from snowflake_feature_store.manager import feature_store_session
from snowflake_feature_store.transforms import fill_na, date_diff

conn = get_connection()

with feature_store_session(conn) as manager:
    
    current_schema = manager.connection.schema
    # Load example data into the feature store schema
    ltv_df, cust_df, behavior_df = get_example_data(
        manager.connection.session,
        current_schema  # Pass the current feature store schema
    )

    # Create feature configurations
    behavior_config, profile_config = create_feature_configs()

    # Create customer entity
    manager.add_entity(
        name="CUSTOMER",
        join_keys=["EMAIL"],
        description="Customer entity for retail domain"
    )

    # Show actual column names
    print("Available columns:", behavior_df.columns)

    # Create feature views with transforms using correct column names
    behavior_view = manager.add_feature_view(
        config=behavior_config,
        df=behavior_df,
        entity_name="CUSTOMER",
        transforms=[fill_na(['AVG_TIME_ON_WEBSITE_MIN'], 0)]  # Column name will be matched case-insensitively
    )


    profile_view = manager.add_feature_view(
        config=profile_config,
        df=cust_df,
        entity_name="CUSTOMER",
        transforms=[
            date_diff('MEMBER_JOIN_DATE', 'MEMBERSHIP_LENGTH_DAYS', 
                    str(datetime.now().strftime('%Y-%m-%d')))
        ]
    )

    # Get training dataset
    training_data = manager.get_features(
        spine_df=ltv_df,
        feature_views=[behavior_config, profile_config],
        label_cols=["LIFE_TIME_VALUE"]
    )

    print("\nTraining data shape:", training_data.count(), len(training_data.columns))
    print("\nFeature columns:", [
        col for col in training_data.columns 
        if col not in ["EMAIL", "LIFE_TIME_VALUE"]
    ])
    training_data.show(2)

Loading example datasets...
Added 1000 customers to table: "CONTAINER_DEMO_DB".FEATURE_STORE_20250211_010928_0478c271.CUSTOMER_LIFE_TIME_VALUE
Added 1000 customers to table: "CONTAINER_DEMO_DB".FEATURE_STORE_20250211_010928_0478c271.CUSTOMER_GENERAL_DATA
Added 1000 customers to table: "CONTAINER_DEMO_DB".FEATURE_STORE_20250211_010928_0478c271.CUSTOMER_BEHAVIOR_DATA

Ltv Data:
-----------------------------------------------------------------
|"EMAIL"               |"CAST (""LIFE_TIME_VALUE"" AS DOUBLE)"  |
-----------------------------------------------------------------
|qisrVQ3yPj@Ue9wz.com  |178.23                                  |
|VFPATq1FiX@nVB6l.com  |339.53                                  |
-----------------------------------------------------------------

Ltv Schema: StructType([StructField('EMAIL', StringType(16777216), nullable=False), StructField('"CAST (""LIFE_TIME_VALUE"" AS DOUBLE)"', DoubleType(), nullable=False)])

Behavior Data:
--------------------------------------

KeyError: 'No StructField named AVG_TIME_ON_WEBSITE_MIN'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()